In [1]:
# Data processing 
import pandas as pd
import numpy as np

import re
import json
from collections import Counter
from itertools import chain

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
#!pip install plotly

In [4]:
# Data vizualizations
import random
import plotly
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
init_notebook_mode(connected=True)
import plotly.offline as offline
import plotly.graph_objs as go

In [5]:
# Data Modeling
from sklearn import model_selection 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [19]:
train_data = pd.read_json('train.json') # store as dataframe objects
test_data = pd.read_json('test.json')

Проанализируем данные в датасете train_data

In [8]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39774 entries, 0 to 39773
Data columns (total 3 columns):
cuisine        39774 non-null object
id             39774 non-null int64
ingredients    39774 non-null object
dtypes: int64(1), object(2)
memory usage: 932.3+ KB


In [9]:
train_data.shape #39774 rows, 3 colls

(39774, 3)

In [10]:
print("The training data consists of {} recipes".format(train_data.shape[0]))

The training data consists of 39774 recipes


In [11]:
print("First five elements in our training sample:")
train_data.head()

First five elements in our training sample:


,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


In [12]:
train_data["ingredients"][0]

['romaine lettuce',
 'black olives',
 'grape tomatoes',
 'garlic',
 'pepper',
 'purple onion',
 'seasoning',
 'garbanzo beans',
 'feta cheese crumbles']

Проанализируем данные в датасете test_data

In [13]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9944 entries, 0 to 9943
Data columns (total 2 columns):
id             9944 non-null int64
ingredients    9944 non-null object
dtypes: int64(1), object(1)
memory usage: 155.5+ KB


In [14]:
test_data.shape

(9944, 2)

In [15]:
print("The training data consists of {} recipes".format(test_data.shape[0]))

The training data consists of 9944 recipes


In [16]:
print("First five elements in our test sample:")
test_data.head()

First five elements in our test sample:


,id,ingredients
0,18009,"[baking powder, eggs, all-purpose flour, raisi..."
1,28583,"[sugar, egg yolks, corn starch, cream of tarta..."
2,41580,"[sausage links, fennel bulb, fronds, olive oil..."
3,29752,"[meat cuts, file powder, smoked sausage, okra,..."
4,35687,"[ground black pepper, salt, sausage casings, l..."


Таким образом, мы имеем тренировочную выборку, содержащую следующие данные:
    id рецепта
    список ингредиентов
    тип кухни, к которому рецепт относится
    
Задача: предсказать тип кухни, к которому относится рецепт. Таким образом, нам нужно решить задачу классификации, которая предварительно требует анализа содержания датасета и создания модели


Рассмотрим более подробно исследуемые данные

In [18]:
for i in train_data.cuisine.unique():
    print(i)

greek
southern_us
filipino
indian
jamaican
spanish
italian
mexican
chinese
british
thai
vietnamese
cajun_creole
brazilian
french
japanese
irish
korean
moroccan
russian


In [19]:
print("Number of cuisine categories: {}".format(len(train_data.cuisine.unique())))

Number of cuisine categories: 20


Мы получили, что у нас больше, чем 2 целевые переменные. 
В данном случае имеет место многоклассовая классификация.

In [20]:
'Simple function for random colours generation'
def random_colours(number_of_colors):
    colors = []
    for i in range (number_of_colors):
        colors.append("#" + ''.join([random.choice('0123456789ABCDEF') for j in range(6)]))
    return colors

Создадим таблицу с инормацией о количестве раз, сколько каждая кухня представлена в тренировочнй выборке

In [21]:
trace = go.Table(
                header = dict(values=['Cuisine','Number of recipes'], fill = dict(color=['#EABEB0']), align = ['left'] * 4),
                         cells = dict(values = [train_data.cuisine.value_counts().index, train_data.cuisine.value_counts()],
                                      align = ['left'] * 4))

layout = go.Layout(title='Number of recipes in each cuisine category',
                   titlefont = dict(size = 20),
                   width=500, height=650, 
                   paper_bgcolor =  'rgba(0,0,0,0)',
                   plot_bgcolor = 'rgba(0,0,0,0)',
                   autosize = False,
                   margin=dict(l=30,r=30,b=1,t=50,pad=1)  )
data = [trace]
fig = dict(data=data, layout=layout)
iplot(fig)

Построим гистограмму с процентным распределением рецептов каждой кухни мира

In [22]:
#  Label distribution in percents
labelpercents = []
for i in train_data.cuisine.value_counts():
    percent = (i/sum(train_data.cuisine.value_counts()))*100
    percent = "%.2f" % percent
    percent = str(percent + '%')
    labelpercents.append(percent)

In [23]:
trace = go.Bar(
            x=train_data.cuisine.value_counts().values[::-1],
            y=[i for i in train_data.cuisine.value_counts().index][::-1],
            text=labelpercents[::-1],  textposition = 'outside', 
            orientation = 'h',marker = dict(color = random_colours(20)))
layout = go.Layout(
                   title='Number of recipes in each cuisine category',
                   titlefont = dict(size = 25),
                   width=1000, height=450, 
                   plot_bgcolor = 'rgba(0,0,0,0)',
                   paper_bgcolor = 'rgba(255, 219, 227, 0.88)',
                   margin=dict(l=75,r=110,b=50,t=60),
                   )
data = [trace]
fig = dict(data=data, layout=layout)
iplot(fig, filename='horizontal-bar')

Из гистограммы видим, что самой популярной кухней является итальянская. Следует за ней Мексиканская и Южноамериканская. Эти рецепты составляют почти 50% всех рецептов.

Проанализируем длину рецептов в тренировочной выборке

In [24]:
print('Maximum Number of Ingredients in a Dish: ',train_data['ingredients'].str.len().max())
print('Minimum Number of Ingredients in a Dish: ',train_data['ingredients'].str.len().min())

Maximum Number of Ingredients in a Dish:  65
Minimum Number of Ingredients in a Dish:  1


In [25]:
longrecipes = train_data[train_data['ingredients'].str.len() > 30]
print("Получается, что {} рецептов состоят из более чем 30 ингредиентов!".format(len(longrecipes)))

Получается, что 40 рецептов состоят из более чем 30 ингредиентов!


In [26]:
print("Рассмотрим самый длинный рецепт в тренировочной выборке:" + "\n")
print(str(list(longrecipes[longrecipes['ingredients'].str.len() == 65].ingredients.values)) + "\n")
print("Рецепт: " + str(list(longrecipes[longrecipes['ingredients'].str.len() == 65].cuisine)))

Рассмотрим самый длинный рецепт в тренировочной выборке:

[['fettucine', 'fresh marjoram', 'minced garlic', 'olive oil', 'garlic powder', 'large eggs', 'Alfredo sauce', 'vegetable oil', 'cajun seasoning', 'shredded romano cheese', 'basil dried leaves', 'salt', 'cayenne pepper', 'scallions', 'red bell pepper', 'boneless skinless chicken breast halves', 'soba', 'pasta sauce', 'kosher salt', 'milk', 'fresh ginger', 'ground black pepper', 'flour', 'cooked chicken', 'coarse salt', 'lemon', 'diced tomatoes', 'garlic', 'rice vinegar', 'NeufchГўtel', 'garlic cloves', 'dried parsley', 'frozen artichoke hearts', 'penne', 'pepper', 'sweet onion', 'part-skim mozzarella cheese', 'parmigiano reggiano cheese', 'basil leaves', 'onion powder', 'red wine vinegar', 'red pepper flakes', 'orzo', 'crushed red pepper', 'all-purpose flour', 'freshly ground pepper', 'sliced mushrooms', 'panko breadcrumbs', 'plum tomatoes', 'fresh basil', 'fresh leav spinach', 'water', 'sun-dried tomatoes', 'ground pepper', 'gr

Исследуем рецепты, состоящие из 1 или 2 ингредиентов

In [27]:
shortrecipes = train_data[train_data['ingredients'].str.len() <= 2]
print("{} рецептов состоят из 1 или 2 продуктов!".format(len(shortrecipes)))

215 рецептов состоят из 1 или 2 продуктов!


In [28]:
print("Исследуем самые короткие рецепты:" + "\n")
print(list(train_data[train_data['ingredients'].str.len() == 1].ingredients.values))

Исследуем самые короткие рецепты:

[['sushi rice'], ['dried rice noodles'], ['plain low-fat yogurt'], ['unsalted butter'], ['udon'], ['sticky rice'], ['butter'], ['corn tortillas'], ['grained'], ['lemonade concentrate'], ['jasmine rice'], ['unsalted butter'], ['cherry tomatoes'], ['butter'], ['cumin seed'], ['haricots verts'], ['vegetable oil'], ['spanish chorizo'], ['sweetened condensed milk'], ['water'], ['phyllo'], ['unsalted butter']]


In [29]:
print("Соответственно, кухни мира, к которым эти рецепты принадлежат" + "\n")
print(list(train_data[train_data['ingredients'].str.len() == 1].cuisine.values))

Соответственно, кухни мира, к которым эти рецепты принадлежат

['japanese', 'vietnamese', 'indian', 'indian', 'japanese', 'thai', 'indian', 'mexican', 'thai', 'southern_us', 'thai', 'indian', 'italian', 'french', 'indian', 'french', 'mexican', 'spanish', 'spanish', 'japanese', 'greek', 'indian']


Таким образом, самый длинный рецепт (65 ингредиентов) принадлежит итальянской кухне.

Самые короткие рецепты принадлежат азиатской культуре - кухням Индии, Японии. 
Некоторые из них, которые содержат только рис вполне возможны, но некоторые рецепты выглядят странно. Например, те, которые состоят только из воды и масла. Здесь уже имеем дело с зашумленными данными или технической ошибкой (в выборку попала только половина рецепта)

In [30]:
#Проианализируем длины рецептов каждой кухни мира
boxplotcolors = random_colours(21)
labels = [i for i in train_data.cuisine.value_counts().index][::-1]
data = []
for i in range(20):
    trace = go.Box(
    y=train_data[train_data['cuisine'] == labels[i]]['ingredients'].str.len(), name = labels[i],
    marker = dict(color = boxplotcolors[i]))
    data.append(trace)
layout = go.Layout(
    title = "Длины рецептов для каждой кухни"
)

fig = go.Figure(data=data,layout=layout)
iplot(fig, filename = "Box Plot Styling Outliers")

Рассмотрим наиболее часто встречающиееся игредиенты в тренировочной выборке.
И те продукты, которые встречаются в выборке только 1 раз.

In [31]:
# список всех ингредиентов (с дубликатами)
allingredients = [] 
for item in train_data['ingredients']:
    for ingr in item:
        allingredients.append(ingr) 

In [32]:
# Словарь, сколько раз каждый ингредиент встречается в выборке 
countingr = Counter()
for ingr in allingredients:
     countingr[ingr] += 1

In [33]:
print("Наиболее часто встречающиеся ингредиенты (первые 20): \n")
print(countingr.most_common(20))
print("\n")

Наиболее часто встречающиеся ингредиенты (первые 20): 

[('salt', 18049), ('onions', 7972), ('olive oil', 7972), ('water', 7457), ('garlic', 7380), ('sugar', 6434), ('garlic cloves', 6237), ('butter', 4848), ('ground black pepper', 4785), ('all-purpose flour', 4632), ('pepper', 4438), ('vegetable oil', 4385), ('eggs', 3388), ('soy sauce', 3296), ('kosher salt', 3113), ('green onions', 3078), ('tomatoes', 3058), ('large eggs', 2948), ('carrots', 2814), ('unsalted butter', 2782)]




In [34]:
print("Общее число уникальных ингредиентов = {}.".format(len(countingr)))

Общее число уникальных ингредиентов = 6714.


In [35]:
# выделим 20 наиболее популярных ингредиентов и построим их гистограмму
mostcommon = countingr.most_common(20)
mostcommoningr = [i[0] for i in mostcommon]
mostcommoningr_count = [i[1] for i in mostcommon]

In [36]:
trace = go.Bar(
            x=mostcommoningr_count[::-1],
            y= mostcommoningr[::-1],
            orientation = 'h',marker = dict(color = random_colours(20),
))
layout = go.Layout(
    xaxis = dict(title= 'Number of occurences in all recipes (training sample)', ),
    yaxis = dict(title='Ingredient',),
    title= '20 Most Common Ingredients', titlefont = dict(size = 20),
    margin=dict(l=150,r=10,b=60,t=60,pad=5),
    width=800, height=500, 
)
data = [trace]
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='horizontal-bar')

Не удивительно, что соль, лук, оливковое масло, перец являются наиболее популярными. сложно представить рецепт без этих компонент. Так что можно сделать вывод, что они не обладают высокой информативностью для предсказания кухни мира, к которой принадлежат рецепты с этими компонентами.

Мы рассмотрели распредeление длин рецептов для каждой кухни. Теперь исследуем ингредиенты, которые специфичны для каждой из кухонь мира

In [37]:
allingredients = list(set(allingredients))

In [38]:
# Define a function that returns a dataframe with top unique ingredients in a given cuisine 
def cuisine_unique(cuisine, numingr, allingredients):
    '''
    Input:
        cuisine - cuisine category (ex. 'brazilian');
        numingr - how many specific ingredients do you want to see in the final result; 
        allingredients - list containing all unique ingredients in the whole sample.
    
    Output: 
        dataframe giving information about the name of the specific ingredient and how many times it occurs in the chosen cuisine (in descending order based on their counts)..

    '''
    allother = []
    for item in train_data[train_data.cuisine != cuisine]['ingredients']:
        for ingr in item:
            allother .append(ingr)
    allother  = list(set(allother ))
    
    specificnonly = [x for x in allingredients if x not in allother]
    
    mycounter = Counter()
    
    for item in train_data[train_data.cuisine == cuisine]['ingredients']:
        for ingr in item:
            mycounter[ingr] += 1
    keep = list(specificnonly)
    
    for word in list(mycounter):
        if word not in keep:
            del mycounter[word]
    
    cuisinespec = pd.DataFrame(mycounter.most_common(numingr), columns = ['ingredient','count'])
    
    return cuisinespec

In [39]:
cuisinespec= cuisine_unique('mexican', 10, allingredients)
print("Top 10 уникальных ингредиентов для мексиканской кухни:")
cuisinespec 

Top 10 уникальных ингредиентов для мексиканской кухни:


,ingredient,count
0,refried beans,250
1,taco seasoning mix,181
2,red enchilada sauce,65
3,taco sauce,63
4,poblano chilies,33
5,mexican chocolate,32
6,baked tortilla chips,31
7,green enchilada sauce,31
8,Mexican beer,30
9,chipotle,25


In [40]:
# Визуализация характерных ингредиентов для первых 10 кухонь
labels = [i for i in train_data.cuisine.value_counts().index][0:10]
totalPlot = 10
y = [[item]*2 for item in range(1,10)]
y = list(chain.from_iterable(y))
z = [1,2]*int((totalPlot/2))

fig = tools.make_subplots(rows= 5, cols=2, subplot_titles= labels, 
                          specs = [[{}, {}],[{}, {}],[{}, {}],[{}, {}],[{}, {}]],  
                          horizontal_spacing = 0.20)
traces = []
for i,e in enumerate(labels): 
    cuisinespec = cuisine_unique(e, 5, allingredients)
    trace = go.Bar(
            x = cuisinespec['count'].values[::-1],
            y = cuisinespec['ingredient'].values[::-1],
            orientation = 'h',marker = dict(color = random_colours(5),))
    traces.append(trace)

for t,y,z in zip(traces,y,z):
    fig.append_trace(t, y,z)

    fig['layout'].update(height=800, width=840,
    margin=dict(l=265,r=5,b=40,t=90,pad=5), showlegend=False, title='Ingredients used only in one cuisine')

iplot(fig, filename='horizontal-bar')

This is the format of your plot grid:
[ (1,1) x1,y1 ]    [ (1,2) x2,y2 ]  
[ (2,1) x3,y3 ]    [ (2,2) x4,y4 ]  
[ (3,1) x5,y5 ]    [ (3,2) x6,y6 ]  
[ (4,1) x7,y7 ]    [ (4,2) x8,y8 ]  
[ (5,1) x9,y9 ]    [ (5,2) x10,y10 ]



In [43]:
#для следующих 10 кухонь
labels = [i for i in train_data.cuisine.value_counts().index][10:20]
totalPlot = 10
y = [[item]*2 for item in range(1,10)]
y = list(chain.from_iterable(y))
z = [1,2]*int((totalPlot/2))

fig = tools.make_subplots(rows= 5, cols=2, subplot_titles= labels, specs = [[{}, {}],[{}, {}],[{}, {}],[{}, {}],[{}, {}]],  horizontal_spacing = 0.20)
traces = []
for i,e in enumerate(labels): 
    cuisinespec= cuisine_unique(e, 5, allingredients)
    trace = go.Bar(
            x= cuisinespec['count'].values[::-1],
            y=  cuisinespec['ingredient'].values[::-1],
            orientation = 'h',marker = dict(color = random_colours(5),))
    traces.append(trace)

for t,y,z in zip(traces,y,z):
    fig.append_trace(t, y,z)

    fig['layout'].update(height=800, width=840,
    margin=dict(l=170,r=5,b=40,t=90,pad=5), showlegend=False, title='Ingredients used only in one cuisine')

iplot(fig, filename='horizontal-bar')

This is the format of your plot grid:
[ (1,1) x1,y1 ]    [ (1,2) x2,y2 ]  
[ (2,1) x3,y3 ]    [ (2,2) x4,y4 ]  
[ (3,1) x5,y5 ]    [ (3,2) x6,y6 ]  
[ (4,1) x7,y7 ]    [ (4,2) x8,y8 ]  
[ (5,1) x9,y9 ]    [ (5,2) x10,y10 ]



__Feature Engineering & Data Modeling__

In [17]:
# Dataset Preparation
print ("Read Dataset ... ")
def read_dataset(path):
    return json.load(open(path)) 

train = read_dataset('train.json')
test = read_dataset('test.json')

Read Dataset ... 


Для решения задачи классификации любой текст необходимо закодировать и представить в виде последовательности чисел.
Выполняем задачу в несколько этапов:
1. приводим рецепты к нижнему регистру и соединяем ингредиенты в предложение
2. используем метод Tfidf-Vectorizer, список предложений пробразуем в матрицу, состоящую из чисел, которые соответствует встречаемости слов в каждом предложении

In [44]:
print ("Приводим тренировочную и тестовую выборки к нижнему регистру ... ")
def generate_text(data):
    text_data = [" ".join(doc['ingredients']).lower() for doc in data]
    return text_data 

Приводим тренировочную и тестовую выборки к нижнему регистру ... 


In [45]:
train_text = generate_text(train)
test_text = generate_text(test)
target = [doc['cuisine'] for doc in train]

In [46]:
# Feature Engineering 
print ("TF-IDF ... ")
tfidf = TfidfVectorizer(binary = True)
def tfidf_features(txt, flag):
    if flag == "train":
        x = tfidf.fit_transform(txt)
    else:
        x = tfidf.transform(txt)
    x = x.astype('float16')
    return x 

TF-IDF ... 


In [47]:
X = tfidf_features(train_text, flag = "train")
X_test = tfidf_features(test_text, flag = "test")

Для преобразования целевой переменной target используем метод LabelEncoding, с помощью которого каждому элементу (в нашем случае, кухне мира) ставится в соответствие число от 0 до 20.

In [48]:
# Label Encoding - Target 
print ("Закодируем целевую переменную тренировочной выборки ... ")
lb = LabelEncoder()
y = lb.fit_transform(target)

Закодируем целевую переменную тренировочной выборки ... 


In [49]:
# Model Training 
print ("Обучим классификатор ... ")
#Support Vector machines Classifier
classifier = SVC(C=100, kernel='rbf', gamma=1, coef0=1,
                 shrinking=True, tol=0.001,  probability=False,  cache_size=200,
                 class_weight = None, verbose=False,  max_iter=-1, decision_function_shape=None,  random_state=None)

model = OneVsRestClassifier(classifier, n_jobs=4)

Обучим классификатор ... 


In [ ]:
model.fit(X, y)

In [ ]:
# Predictions 
print ("Сделаем предсказание на тестовой выборке ... ")
y_test = model.predict(X_test)
y_pred = lb.inverse_transform(y_test)

In [ ]:
# Submission
print ("Сгенерируем файл с ответами ... ")
test_id = [doc['id'] for doc in test]
sub = pd.DataFrame({'id': test_id, 'cuisine': y_pred}, columns=['id', 'cuisine'])
sub.to_csv('svm_output.csv', index=False)

In [20]:
test_data.head()

,id,ingredients
0,18009,"[baking powder, eggs, all-purpose flour, raisi..."
1,28583,"[sugar, egg yolks, corn starch, cream of tarta..."
2,41580,"[sausage links, fennel bulb, fronds, olive oil..."
3,29752,"[meat cuts, file powder, smoked sausage, okra,..."
4,35687,"[ground black pepper, salt, sausage casings, l..."


In [13]:
result = pd.read_csv('svm_output.csv', sep = ',')

In [14]:
result.head()

,id,cuisine
0,18009,irish
1,28583,southern_us
2,41580,italian
3,29752,cajun_creole
4,35687,italian
